![Model Training](https://drive.google.com/uc?id=17cUxpbL2eonLzgY0uIVAxmsfMfQF1by2)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls '/content/drive/My Drive/Colab Notebooks/twitter.properties'

In [ ]:
!pip install ConfigParser

[twitter]
accesstoken=xxxx

accesstokensecret=xxxx

apikey=xxxxxx

apisecretkey=xxxxxx

googleapikey=xxxxxxx

In [ ]:
import pandas as pd
import configparser
config = configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/twitter.properties')

print(config.sections());


In [ ]:
corona_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')

In [ ]:
corona_confirmed_df.head()

In [ ]:
corona_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')

In [ ]:
corona_death_df.head()

In [ ]:
corona_recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [ ]:
corona_recovered_df.head()

In [ ]:
corona_confirmed_df=corona_confirmed_df.melt(id_vars=['Province/State','Country/Region','Lat','Long'])

In [ ]:
corona_confirmed_df.head()

In [ ]:
corona_confirmed_df=corona_confirmed_df.rename({'variable':'Date','value': 'Confirmed'}, axis='columns')

In [ ]:
corona_confirmed_df.head()

In [ ]:
corona_death_df=corona_death_df.melt(id_vars=['Province/State','Country/Region','Lat','Long'])

In [ ]:
corona_death_df=corona_death_df.rename({'variable':'Date','value': 'Death'}, axis='columns')

In [ ]:
corona_death_df.head()

In [ ]:
corona_recovered_df=corona_recovered_df.melt(id_vars=['Province/State','Country/Region','Lat','Long']).rename({'variable':'Date','value': 'Recovered'}, axis='columns')

In [ ]:
corona_recovered_df.head()

In [ ]:
combined_df = [corona_confirmed_df, corona_death_df, corona_recovered_df]
combined_df = [df.set_index(['Province/State','Country/Region','Lat','Long','Date']) for df in combined_df]
combined_df=combined_df[0].join(combined_df[1:])

In [ ]:
combined_df.head()

In [ ]:
combined_df=combined_df.reset_index()

In [ ]:
combined_df.head()

In [ ]:
combined_df[['Lat', 'Long','Confirmed','Death','Recovered']] = combined_df[['Lat', 'Long','Confirmed','Death','Recovered']].apply(pd.to_numeric) 

In [ ]:
combined_df[['Date']] = combined_df[['Date']].apply(pd.to_datetime)

In [ ]:
combined_df.dtypes

In [ ]:
combined_df.head()

In [ ]:

combined_df=combined_df.rename({'Province/State':'State','Country/Region': 'Country'}, axis='columns')

In [ ]:
combined_df.head()

In [ ]:
combined_df.isnull().any()

In [ ]:
combined_df.count()

In [ ]:
combined_df.set_index('Country').filter(like="India", axis=0)

In [ ]:
combined_df[combined_df.State.isnull()]['Country'].value_counts()

In [ ]:
!pip install -U googlemaps

In [ ]:
import googlemaps

In [ ]:
gmaps = googlemaps.Client(key=config.get('twitter', 'googleapikey'))

geocode_result = gmaps.reverse_geocode((-23.4425, -58.4438))

print(geocode_result[0])
print(geocode_result[0]['formatted_address'])
print(geocode_result[0]['formatted_address'].split(",")[-2].strip(" "))

In [ ]:
import re

english_check = re.compile(r'[a-z]')

def get_state(lat, longi):
  try:
    output=gmaps.reverse_geocode((lat, longi))[0]['formatted_address'].split(",")[-2].strip()
    if(len(output.split(" ")) > 1):
      output="TBF"

    if not english_check.match(output.lower()):
      output="TBF"
    
  except:
    print("Error in Reverse Geocoding")
    output="TBF"
  return output

In [ ]:
combined_df["state_cleaned"]=combined_df[combined_df.State.isnull()][['Lat','Long']].apply(lambda x : get_state(x['Lat'], x['Long']),axis=1)


In [ ]:
combined_df[combined_df.state_cleaned.isnull()]['Country'].value_counts()

In [ ]:
combined_int_df = combined_df.copy()

In [ ]:
combined_df[combined_df.state_cleaned.notna()]

In [ ]:
combined_int_df.count()

In [ ]:
combined_int_df[combined_int_df.State.notna()]

In [ ]:
combined_int_df[combined_int_df.State.notna()]["State"].count()

In [ ]:
import numpy
combined_int_df["state_cleaned"]=combined_int_df.apply(lambda x: x['State'] if (numpy.all(pd.notnull(x['State']))) else x['state_cleaned'], axis=1)

In [ ]:
combined_int_df[combined_int_df.State.notna()]

In [ ]:
combined_int_df[combined_df.state_cleaned=="TBF"]

In [ ]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [ ]:
combined_int_df.head(50)

In [ ]:
capital_df = pd.read_csv('https://raw.githubusercontent.com/icyrockcom/country-capitals/master/data/country-list.csv')

In [ ]:
capital_df.head()

In [ ]:
capital_df['country'].replace({"United Kingdom; England": "United Kingdom"}, inplace=True)

In [ ]:
capital_df['country']=capital_df.country.str.lower()

In [ ]:
capital_df.head()

In [ ]:
capital_df=capital_df.set_index('country')

In [ ]:
capital_df.loc['algeria'][0]

In [ ]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]

In [ ]:
def lookup_capital(country):
  try:
    capital=capital_df.loc[country.lower()][0]
  except:
    capital="TBF"
  return capital  

In [ ]:
combined_int_df.apply(lambda x: lookup_capital(x['Country']) if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [ ]:
combined_int_df['state_cleaned']=combined_int_df.apply(lambda x: lookup_capital(x['Country']) if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [ ]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [ ]:
combined_int_df["Country"].replace({"Republic of Moldova": "Moldova", "Holy See": "Vatican City", "UK": "United Kingdom","Viet Nam":"Vietnam","Iran (Islamic Republic of)":"Iran"}, inplace=True)

In [ ]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [ ]:
combined_int_df['state_cleaned']=combined_int_df.apply(lambda x: lookup_capital(x['Country']) if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [ ]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [ ]:
combined_int_df["state_cleaned"]=combined_int_df.apply(lambda x: x['Country'] if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [ ]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

In [ ]:
combined_int_df.to_csv('corona_dataset.csv')

In [ ]:
combined_int_df[combined_int_df.Country=="US"]

In [ ]:
usstates_df = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')

In [ ]:
usstates_df.head()

In [ ]:
usstates_df=usstates_df.set_index('Abbreviation')

In [ ]:
usstates_df.head()

In [ ]:
usstates_df.loc['AZ']

In [ ]:
combined_int_df['City']=combined_int_df.apply(lambda x: x['State'].split(",")[0] if(x['Country']=="US") else "", axis=1)

In [ ]:
def convert_state(state):
  try:
    output=state.split(",")[1].strip()
    output=usstates_df.loc[output.upper()][0]
  except:
    output=state
  return output  

In [ ]:
combined_int_df["state_cleaned"]=combined_int_df.apply(lambda x: convert_state(x['state_cleaned']) if(x['Country']=="US") else x['state_cleaned'], axis=1)

In [ ]:
combined_int_df.head(50)

In [ ]:
combined_int_df[combined_int_df.Country=="US"][0:100]

In [ ]:
combined_int_df.to_csv("corona_dataset.csv")

In [ ]:
combined_int_df.apply(lambda x: 1 if(x['Country']=="US" and numpy.all(pd.notnull(x['City']))) else 0, axis=1)